In [ ]:
from datetime import datetime
from pprint import pprint as pp

import pandas as pd
import pygwalker as pyg
from api_binance import get_klines
from api_bitpreco import (
    Balance,
    OpenOrders,
    Orderbook,
    Ticker,
    Trades,
    get_coinpair,
)
from rich import print

# Tabela da ordem de compra e venda executadas
from segredos import CAMINHO


In [ ]:
print(datetime.now().astimezone().timetz().tzinfo)
# dois dias atrás em timestamp
print(datetime.now().timestamp() - 2 * 24 * 60 * 60)

In [ ]:
response = get_klines()

display(response.columns)

# Minhas Ordens executadas

In [ ]:
coinpair = get_coinpair()
coinpair = 'all-brl'
pp(Ticker(coinpair).json().keys())

In [ ]:
balance = Balance().json()
# pp(balance)
balance_df = pd.DataFrame([balance])

balance_df

In [ ]:
open_orders = OpenOrders().json()

if open_orders != []:
    open_orders_df = pd.DataFrame(open_orders)
    pyg.walk(open_orders_df)

In [ ]:
def calculate_fee(
    order_type: str, origin: str, amount: float, price: float
) -> float:
    """
    Calcula a taxa da ordem baseado no tipo, origem e valor.

    Args:
        order_type: Tipo da ordem ('BUY' ou 'SELL')
        origin: Origem da ordem ('API', 'APP' ou 'WEB')
        amount: Quantidade de BTC
        price: Preço unitário em BRL

    Returns:
        float: Valor da taxa em BTC
    """
    valor_total = amount * price

    # Taxa para ordens de venda (SELL)
    if order_type == 'SELL':
        if origin in {'API', 'WEB'}:
            return amount * 0.002  # 0.2%
        elif origin == 'APP':
            return amount * 0.002  # 0.2%

    # Taxa para ordens de compra (BUY)
    elif order_type == 'BUY':
        if origin == 'API':
            return 2e-8 if valor_total < 50 else 5e-7  # noqa: PLR2004
        elif origin == 'APP':
            return 0.0  # Algumas ordens APP têm taxa zero
        elif origin == 'WEB':
            return 0.0  # Algumas ordens WEB têm taxa zero

    return 0.0

In [ ]:
ORDERS_FILE = CAMINHO + '/executed_orders.csv'

executed_orders_df = pd.read_csv(ORDERS_FILE)
# pegando todas as ordens de compra até a ultima ordem de venda
# Filtra ordens executadas até a última ordem de venda
last_sell_index = executed_orders_df[
    executed_orders_df['type'] == 'SELL'
].index[0]
filtered_orders = executed_orders_df.loc[:last_sell_index]

# Filtra apenas ordens de compra
buy_orders = filtered_orders[filtered_orders['type'] == 'BUY']
print(executed_orders_df)
print(buy_orders)

# criando uma lista de amount que ainda precisam ser
# vendidos com seus respectivos preços e taxas
amounts_to_sell = []


Preciso de um grafico que me mostre o melhor momento para comprar bitcoin

In [ ]:
orders_json = Orderbook().json()

pp(orders_json)

In [ ]:
# transformando em um dataframe pandas para ver no pygwalker
df = pd.DataFrame(orders_json)

# Expandir as colunas 'bids' e 'asks' em DataFrames separados
bids_df = pd.DataFrame(orders_json['bids'])
asks_df = pd.DataFrame(orders_json['asks'])

# Adicionar uma coluna para indicar o tipo de ordem
bids_df['type'] = 'bid'
asks_df['type'] = 'ask'

# Concatenar os DataFrames
orders_df = pd.concat([bids_df, asks_df], ignore_index=True)

# Exibir o DataFrame resultante
display(orders_df)

In [ ]:
pyg.walk(orders_df)

### Retorna uma lista de ordens já executadas.

Os retornos terão a seguinte estrutura:

type - Se a ordem é do tipo compra ou do tipo venda;
amount - Volume negociado na ordem;
market - Moedas envolvidas na ordem;
price - Preço do BTC no momento da ordem;
timestamp - Timestamp da ordem.

In [ ]:
trades_df = pd.DataFrame(Trades().json())

pyg.walk(trades_df)

In [ ]:
# import plotly.graph_objects as go

# fig = go.Figure(
#     data=[
#         go.Candlestick(
#             x=data['timestamp'],
#             open=data['open'],
#             high=data['high'],
#             low=data['low'],
#             close=data['close'],
#         )
#     ]
# )
# fig.show()